# Assignment 2: Normalizing Data

> This assignment will involve data from the (attached) `enrollments.csv` file.
> 
> Please write all of your answers for this assignment within this Jupyter notebook. Submit the completed notebook on Canvas.
> - You may work on the CS1 server, your own device, or wherever else is convenient.
> 
> - You will need to complete problems 4-6 prior to problems 7-10.
> 
> Run your SQL code from Python using the `psycopg2` library. An example of this is provided below; your code answers should look similar.

Joshua D. Ingram

10/12/2022

Note: Worked on some problems together with Fehmi.

In [2]:
import psycopg2

# For reading query output into a pandas dataframe
import pandas as pd
import pandas.io.sql as sqlio

# Connect to the database
conn = psycopg2.connect("dbname=test")


In [3]:
# Reset any prior transaction
conn.rollback()

# Create a cursor
cur = conn.cursor()

# Insert some temporary data
cur.execute("""
DROP TABLE IF EXISTS demo;

CREATE TEMPORARY TABLE demo (
    id SERIAL PRIMARY KEY,
    x INT,
    y INT
);

INSERT INTO demo (x, y) VALUES
    (1, 2),
    (3, 4),
    (5, 6);
""")

# Read into a dataframe for pretty-printing
df = sqlio.read_sql_query("""
SELECT *, x+y AS Z FROM demo
""", conn)

conn.commit()

# Print the dataframe
# (This may also display a warning)
print(df)

   id  x  y   z
0   1  1  2   3
1   2  3  4   7
2   3  5  6  11


/Users/joshuaingram/opt/anaconda3/envs/dbs/lib/python3.10/site-packages/pandas/io/sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


> Note that you may have to call `conn.rollback()` if one of your commands fails. See the [docs](https://www.psycopg.org/docs/) for more examples and information.

> ## Problem 1.
>
> What is a functional dependency? Please explain, in your own words.
> 
> How can you test whether a functional dependency exists between two attributes? Give at least two examples of functional dependencies.

A functional dependency is a relation between two attributes, with one attribute being the determinant of the value of the other attribute, which is the dependent.

To test the functional dependency between two attributes we can change the value of one attribute and see if it necessarily changes the value of the other, and that this attribute is unique (e.g. student_id can change, but ). We check for one way dependency. 
Examples:
(user_id) -> (account_creation_date)
(SSN) -> (date_of_birth)

> ## Problem 2.
> Identify all of the functional dependencies in the `enrollments.csv` data.
> 
> Write them in a list.
> 
> Example:
>> - (Social Security Number) -> (Date of Birth)
>> - (Species) -> (Genus, Order, Phylum)
>> - (Genus) -> (Order, Phylum)
>> - (Order ID) -> (Customer ID)

- (student_id) -> (name, email, year_enrollment)
- (semester_label) <-> (academic_year, term)
- (student_id, semester_label) -> (major)
- (course_dept, course_number) -> (course_title) 
- (course_dept, course_number, semester_label) -> (professor)
- (student_id, course_dept, course_number, semester_label) -> (grade) [students can have multiple grades even with all attributes the same...]

> ## Problem 3.
> Is the enrollments data in a normal form? For each of the three normal forms we've discussed (First, Second, and Third), explain why it is or is not in that normal form.

Yes, it is in 1NF.

First Normal Form
- Each of the rows are unique, within each row there are no attributes with multiple values.

Second Normal Form
- It is not in second normal form. Although the enrollments data is in 1NF, there are partial dependencies. The primary key (composite key) is made up of (student_id, course_dept, course_number, semester_label). There exists a partial dependency, for example, with the major which only depends on the semester_label and student_id.

Third Normal Form
- Not in third normal form because it does not fulfill second normal form. There can be no transitive dependency.

>## Problem 4.
>
>### (4a)
>
>Write down a schema for the `enrollments.csv` data that is in third normal form.
>- Each column from the spreadsheet must be placed into some table.
>- You may add new surrogate keys and foreign keys.
>- For convenience, you may add a new column for "semester number".
>
>Indicate which attributes are keys and which must be unique or non-null.
>
> For example, given a hypothetical `contacts.csv` with columns `ssn,dob,name,phone_number`, we might produce:
> 
>> Person
>> - Person ID (PK)
>> - SSN (unique, not null)
>> - Date of Birth (not null)
>>
>> Contact Info
>> - Contact Info ID (PK)
>> - Person ID (FK, not null)
>> - Name (not null)
>> - Phone number

Students
- student_id ( pk)
- name (not null)
- email (unique, not null)
- year_enrolled (not null)

Courses
- course_id (pk)
- course_dept ( not null)
- course_number (not null)
- course_title (not null)

Professors
- course_id (pk, fk)
- semester_label (pk, fk)
- professor (not null)

Majors
- semester_label (pk, fk)
- student_id (pk, fk)
- major (unique, not null)

Semesters
- semester_label (pk)
- academic_year (unique, not null)
- term (not null)

Grades
- semester_label (pk, fk)
- student_id (pk, fk)
- course_id (pk, fk)
- grade (float(20),not null)

> ## (4b)
>
>Produce an Entity-Relational diagram for your schema. You may hand-draw your diagram, or use a tool such as <dbdiagram.io>.
>
>Clearly indicate which relationships are one-to-one, one-to-many, many-to-many, etc.

![](diagram.jpeg)

Please not that there should NOT be a branch from students to semesters... For some reason the photo in the notebook is not updating, even when I added the new photo to the path.

>## (4c)
>
> Based on the foreign-key relationships, write down the order in which your tables will need to be created.
>
> Example:
>> 1. Person
>> 2. Contact Info

1. Students
2. Courses
3. Semesters
4. Majors
5. Professors
6. Grades

Note: some can be reordered.

> ## Problem 5.
> Write the SQL to create your tables.
> - Make sure to wrap it in a transaction.
> - Use the order from problem 4c.
> - Include all relevant constraints.
> 
> Example:
>>```sql
>>BEGIN;
>>
>>CREATE TABLE person (
>>    person_id SERIAL PRIMARY KEY,
>>    ssn TEXT UNIQUE NOT NULL,
>>    dob DATE NOT NULL
>>);
>>
>>CREATE TABLE contact_info (
>>    contact_info_id SERIAL PRIMARY KEY,
>>    person_id INTEGER NOT NULL REFERENCES person(person_id),
>>    name TEXT NOT NULL,
>>    phone_number TEXT
>>);
>>
>>COMMIT;
>>
>>```

In [2]:
BEGIN;

CREATE TABLE STUDENTS (
    STUDENT_ID TEXT PRIMARY KEY,
    name TEXT NOT NULL,
    EMAIL TEXT UNIQUE NOT NULL,
    YEAR_ENROLLED INTEGER NOT NULL
);
CREATE TABLE COURSES (
    COURSE_ID SERIAL PRIMARY KEY,
    COURSE_DEPT TEXT NOT NULL,
    COURSE_NUMBER INTEGER NOT NULL,
    COURSE_TITLE TEXT NOT NULL
);
CREATE TABLE SEMESTERS (
    SEMESTER_LABEL TEXT PRIMARY KEY,
    ACADEMIC_YEAR INTEGER NOT NULL,
    TERM INTEGER NOT NULL
);
CREATE TABLE MAJORS (
    SEMESTER_LABEL TEXT references SEMESTERS(SEMESTER_LABEL),
    STUDENT_ID TEXT references STUDENTS(STUDENT_ID),
    MAJOR TEXT NOT NULL,
    PRIMARY KEY (SEMESTER_LABEL, STUDENT_ID)
);
CREATE TABLE PROFESSORS (
    PROFESSOR_ID SERIAL PRIMARY KEY,
    SEMESTER_LABEL TEXT references SEMESTERS(SEMESTER_LABEL),
    COURSE_ID INTEGER NOT NULL references COURSES(COURSE_ID),
    PROFESSOR TEXT NOT NULL
);
CREATE TABLE GRADES (
    SEMESTER_LABEL TEXT references SEMESTERS(SEMESTER_LABEL),
    STUDENT_ID TEXT references STUDENTS(STUDENT_ID),
    COURSE_ID INTEGER references COURSES(COURSE_ID),
    GRADE FLOAT(20) NOT NULL,
    PRIMARY KEY (SEMESTER_LABEL, STUDENT_ID, COURSE_ID)
);

COMMIT;

SyntaxError: invalid syntax (3236006541.py, line 3)

> ## Problem 6.
> Write the SQL to import and normalize the `enrollments.csv` data.
> - Begin by copying it from file to a temporary table.
> - Populate your normalized tables in the same order as problems 4c and 5.
> - Make sure to wrap everything in a transaction.
> 
> Example:
>> ```sql
>> BEGIN;
>> 
>> CREATE TEMPORARY TABLE data (
>>     ssn TEXT,
>>     dob DATE,
>>     name TEXT,
>>     phone_number TEXT
>> );
>> 
>> COPY data FROM 'data.csv' WITH CSV HEADER;
>> 
>> -- Begin normalizing
>> INSERT INTO person (ssn, dob)
>> SELECT DISTINCT ssn, dob
>> FROM data;
>> 
>> -- Join to get the newly generated keys
>> WITH joined AS (
>>     SELECT person_id, data.*
>>     FROM person JOIN data
>>     ON person.ssn=data.ssn
>>     AND person.dob = data.dob
>> ) INSERT INTO contact_info (person_id, name, phone_number)
>> SELECT person_id, name, phone_number
>> FROM joined;
>> 
>> COMMIT;
>> ```

In [3]:
BEGIN;

CREATE TEMPORARY TABLE DATA (
    STUDENT_ID TEXT,
    NAME TEXT,
    EMAIL TEXT,
    YEAR_ENROLLED INTEGER,
    MAJOR TEXT,
    COURSE_DEPT TEXT,
    COURSE_NUMBER INTEGER,
    COURSE_TITLE TEXT,
    PROFESSOR TEXT,
    ACADEMIC_YEAR INTEGER,
    TERM INTEGER,
    SEMESTER_LABEL TEXT,
    GRADE FLOAT
);

COPY DATA FROM '/Users/joshuaingram/Main/Projects/masters_coursework/fall_2022/databases_for_data_science/assignments/assignment2/enrollment.csv' WITH CSV HEADER;

INSERT INTO STUDENTS (STUDENT_ID, NAME, EMAIL, YEAR_ENROLLED)
SELECT DISTINCT STUDENT_ID, NAME, EMAIL, YEAR_ENROLLED
FROM DATA;

INSERT INTO COURSES (COURSE_DEPT, COURSE_NUMBER, COURSE_TITLE)
SELECT DISTINCT COURSE_DEPT, COURSE_NUMBER, COURSE_TITLE
FROM DATA;

INSERT INTO SEMESTERS (SEMESTER_LABEL, ACADEMIC_YEAR, TERM)
SELECT DISTINCT SEMESTER_LABEL, ACADEMIC_YEAR, TERM
FROM DATA;

INSERT INTO MAJORS (SEMESTER_LABEL, STUDENT_ID, MAJOR)
SELECT DISTINCT SEMESTER_LABEL, STUDENT_ID, MAJOR
FROM DATA;

-- HAD TO USE SERIAL FOR PROFESSOR_ID SINCE THERE CAN BE MULTIPLE PROFESSORS TEACHING THE SAME CLASS IN THE SAME SEMESTER
WITH JOINED AS (
    SELECT COURSE_ID, DATA.*
    FROM COURSES JOIN DATA
    ON COURSES.COURSE_DEPT = DATA.COURSE_DEPT
    AND COURSES.COURSE_NUMBER = DATA.COURSE_NUMBER
    AND COURSES.COURSE_TITLE = DATA.COURSE_TITLE
) INSERT INTO PROFESSORS (COURSE_ID, SEMESTER_LABEL, PROFESSOR)
SELECT DISTINCT COURSE_ID, SEMESTER_LABEL, PROFESSOR
FROM JOINED;

WITH JOINED AS (
    SELECT COURSE_ID, DATA.*
    FROM COURSES JOIN DATA
    ON COURSES.COURSE_DEPT = DATA.COURSE_DEPT
    AND COURSES.COURSE_NUMBER = DATA.COURSE_NUMBER
    AND COURSES.COURSE_TITLE = DATA.COURSE_TITLE 
) INSERT INTO GRADES (SEMESTER_LABEL, COURSE_ID, STUDENT_ID, GRADE)
SELECT DISTINCT SEMESTER_LABEL, COURSE_ID, STUDENT_ID, GRADE
FROM JOINED;

COMMIT;

SyntaxError: invalid syntax (2081681561.py, line 3)

In [ ]:
WITH JOINED AS (
    SELECT COURSE_ID, DATA.*
    FROM COURSES JOIN DATA
    ON COURSES.COURSE_DEPT = DATA.COURSE_DEPT
    AND COURSES.COURSE_NUMBER = DATA.COURSE_NUMBER
    AND COURSES.COURSE_TITLE = DATA.COURSE_TITLE
) INSERT INTO PROFESSORS (COURSE_ID, SEMESTER_LABEL, PROFESSOR)
SELECT DISTINCT COURSE_ID, SEMESTER_LABEL, PROFESSOR
FROM JOINED;

> # Note.
>
> In the following questions, you will query the freshly normalized data.
> 
> If you are asked to "write a query", please produce a *single* SQL command. (That is, there should only be one `;`.)
> 
> You should use subqueries (`FROM (...) my_alias`) or common table expressions (`WITH my_alias AS (...)`), but do not create tables or views unless explicitly instructed to do so.
> 
> You must use the normalized tables from problems 5 and 6, not the initial denormalized CSV format.

> ## Problem 7.
>
> ### (7a)
>
> Write a query that shows, for each student, the number of semesters in which they were enrolled.

In [33]:
SELECT
    COUNT(ENROLLMENT.SEMESTER_LABEL),
    STUDENT_ID
FROM
    (
    SELECT DISTINCT
        STUDENT_ID, SEMESTER_LABEL
    FROM
        GRADES
    ) ENROLLMENT
GROUP BY STUDENT_ID;

> ### (7b)
>
> Create a view that shows the total number of students enrolled per academic year.
> - Make sure each student is counted once, and only once, per year.
> - Note that, for example, the academic year 1995 would consist of the FA1995 and SP1996 semesters.

In [34]:
-- NOTE: the academic_year column includes the fall of that year and spring of the next in semester label. Makes this much easier.
CREATE VIEW enrolled_yearly AS
    SELECT
        COUNT(DISTINCT AC_YEAR.STUDENT_ID) AS NUM_ENROLLED,
        AC_YEAR.ACADEMIC_YEAR
    FROM (
        SELECT
            ACADEMIC_YEAR,
            STUDENT_ID
        FROM
            SEMESTERS
            JOIN GRADES ON SEMESTERS.SEMESTER_LABEL = GRADES.SEMESTER_LABEL
    ) AC_YEAR
    GROUP BY AC_YEAR.ACADEMIC_YEAR;

> ### (7c)
> 
> Write a query that shows the number of *first-year* students majoring in MATH in each academic year.
> - You can include students who change their major within the year.

In [35]:
SELECT
    COUNT(DISTINCT STUDENT_MAJORS.STUDENT_ID) AS COUNT,
    STUDENT_MAJORS.MAJOR,
    STUDENT_MAJORS.ACADEMIC_YEAR
FROM (
    SELECT
        STUDENTS.STUDENT_ID,
        STUDENTS.YEAR_ENROLLED,
        MAJOR_AC_YEAR.ACADEMIC_YEAR,
        MAJOR_AC_YEAR.MAJOR
    FROM (
        SELECT
            MAJORS.SEMESTER_LABEL,
            ACADEMIC_YEAR,
            STUDENT_ID,
            MAJOR
        FROM
            MAJORS
            JOIN SEMESTERS ON SEMESTERS.SEMESTER_LABEL = MAJORS.SEMESTER_LABEL
    ) MAJOR_AC_YEAR
    JOIN STUDENTS ON MAJOR_AC_YEAR.STUDENT_ID = STUDENTS.STUDENT_ID
) STUDENT_MAJORS
WHERE STUDENT_MAJORS.YEAR_ENROLLED = STUDENT_MAJORS.ACADEMIC_YEAR AND STUDENT_MAJORS.MAJOR = 'MATH'
GROUP BY STUDENT_MAJORS.MAJOR, STUDENT_MAJORS.ACADEMIC_YEAR;

> ### Bonus - optional
>
> Convert your query from 7c into a function. The function should take a major and a student-year, and return the number of those students with that major in each academic year.
> - e.g., (PHIL, 3) -> (# of third-year philosophy students each year).

In [36]:
CREATE FUNCTION MAJOR_COUNT(MAJOR_DESIRED TEXT, STUDENT_YEAR INTEGER)
RETURNS TABLE (
    COUNT INTEGER,
    MAJOR TEXT,
    YEAR INTEGER
)
AS 
$$
BEGIN
SELECT
    COUNT(DISTINCT STUDENT_MAJORS.STUDENT_ID) AS COUNT,
    STUDENT_MAJORS.MAJOR,
    STUDENT_MAJORS.ACADEMIC_YEAR
FROM (
    SELECT
        STUDENTS.STUDENT_ID,
        STUDENTS.YEAR_ENROLLED,
        MAJOR_AC_YEAR.ACADEMIC_YEAR,
        MAJOR_AC_YEAR.MAJOR
    FROM (
        SELECT
            MAJORS.SEMESTER_LABEL,
            ACADEMIC_YEAR,
            STUDENT_ID,
            MAJOR
        FROM
            MAJORS
            JOIN SEMESTERS ON SEMESTERS.SEMESTER_LABEL = MAJORS.SEMESTER_LABEL
    ) MAJOR_AC_YEAR
    JOIN STUDENTS ON MAJOR_AC_YEAR.STUDENT_ID = STUDENTS.STUDENT_ID
) STUDENT_MAJORS
WHERE STUDENT_MAJORS.YEAR_ENROLLED + (MAJOR_COUNT.STUDENT_YEAR-1) = STUDENT_MAJORS.ACADEMIC_YEAR AND STUDENT_MAJORS.MAJOR = MAJOR_DESIRED
GROUP BY STUDENT_MAJORS.MAJOR, STUDENT_MAJORS.ACADEMIC_YEAR;
END;
$$ LANGUAGE PLPGSQL;

> ## Problem 8.
> 
> ### (8a)
> 
>Write a query that returns the number of professors and courses per department.
> 
> Example output:
> 
>> ```csv
>> dept,n_professors,n_courses
>> ASTR,2,9
>> ETYM,1,4
>> ```


In [ ]:
SELECT
    COURSE_DEPT AS DEPT,
    COUNT(DISTINCT PROF_DEPT.PROFESSOR) AS N_PROFESSORS,
    COUNT(DISTINCT PROF_DEPT.COURSE_ID) AS N_COURSES
FROM (
    SELECT DISTINCT
        PROFESSORS.PROFESSOR,
        COURSES.COURSE_ID,
        COURSES.COURSE_DEPT
    FROM
        COURSES
        JOIN PROFESSORS ON PROFESSORS.COURSE_ID = COURSES.COURSE_ID
) PROF_DEPT
GROUP BY COURSE_DEPT;

> ### (8b)
>
> Write a query that returns, for each professor, the number of distinct students they taught in each semester.
> 
> Example output:
> 
>> ```csv
>> professor,semester,n_students
>> Gabriel Medici,FA2002,31

In [ ]:
SELECT
    PROFESSOR,
    SEMESTER_LABEL,
    COUNT(DISTINCT STUDENT_ID) AS N_STUDENTS
FROM (
    SELECT DISTINCT
        PROFESSORS.PROFESSOR,
        GRADES.COURSE_ID,
        GRADES.SEMESTER_LABEL,
        GRADES.STUDENT_ID
    FROM
        GRADES
        JOIN PROFESSORS ON PROFESSORS.COURSE_ID = GRADES.COURSE_ID
) PROF_STUDENTS
GROUP BY PROFESSOR, SEMESTER_LABEL;

> ### (8c)
> 
> Write a query that returns, for each professor, the number of distinct students they taught across all semesters.
> 
> Include an additional column indicating how many of these students majored in that professor's department.
>
> Example output: 
>> ```csv
>> professor, dept, n_students, n_students_in_dept
>> Gabriel Medici,ASTR,379,255
>> ```

In [1]:
SELECT
    PROFESSOR,
    COURSE_DEPT,
    COUNT(DISTINCT STUDENT_COURSE_DEPT.STUDENT_ID) AS N_STUDENTS,
    COUNT(DISTINCT CASE WHEN MAJOR = COURSE_DEPT THEN STUDENT_COURSE_DEPT.STUDENT_ID END) AS N_STUDENTS_IN_DEPT
FROM (
    SELECT DISTINCT
    PROF_DEPT.PROFESSOR,
    PROF_DEPT.COURSE_DEPT,
    GRADES.COURSE_ID,
    GRADES.SEMESTER_LABEL,
    GRADES.STUDENT_ID
FROM (
    SELECT DISTINCT
        PROFESSORS.PROFESSOR,
        COURSES.COURSE_ID,
        COURSES.COURSE_DEPT
    FROM
        COURSES
        JOIN PROFESSORS ON PROFESSORS.COURSE_ID = COURSES.COURSE_ID
) PROF_DEPT
    JOIN GRADES ON PROF_DEPT.COURSE_ID = GRADES.COURSE_ID
) STUDENT_COURSE_DEPT
    JOIN MAJORS ON STUDENT_COURSE_DEPT.STUDENT_ID = MAJORS.STUDENT_ID
GROUP BY PROFESSOR, COURSE_DEPT;


IndentationError: unexpected indent (3856273134.py, line 2)

> ## Problem 9.
> 
> ### (9a)
> 
> Write a query that returns each student's average grade per semester.

In [ ]:
SELECT
    SEMESTER_LABEL,
    STUDENT_ID,
    AVG(GRADE) AS AVERAGE_GRADE
FROM
    GRADES
GROUP BY SEMESTER_LABEL, STUDENT_ID;

> ### (9b)
> 
> Write a query that returns the average grade for each course (across all semesters in which the course was taught).
>
> Example output: 
>> ```csv
>> dept,course_number,course_title,avg_grade
>> CSCI,3500,Cellular Automata,0.79
>> ```


In [ ]:
SELECT
    COURSE_DEPT,
    COURSE_NUMBER,
    COURSE_TITLE,
    AVG(GRADES.GRADE) AS AVERAGE_GRADE
FROM
    GRADES
    JOIN COURSES ON COURSES.COURSE_ID = GRADES.COURSE_ID
GROUP BY COURSE_DEPT, COURSE_NUMBER, COURSE_TITLE


> ### (9c)
> 
> Write a query that returns the minimum, maximum, and average class size for each course (across all semesters).
>
> Example output: 
>> ```csv
>> dept,course_number,course_title,min_size,max_size,avg_size
>> CSCI,3500,Cellular Automata, 12,41,23.3
>> ```

In [ ]:
SELECT
    COURSE_DEPT,
    COURSE_NUMBER,
    COURSE_TITLE,
    MIN(COUNT) AS MIN_SIZE,
    MAX(COUNT) AS MAX_SIZE,
    AVG(COUNT) AS AVG_SIZE
FROM (
    SELECT
        COURSE_DEPT,
        COURSE_NUMBER,
        COURSE_TITLE,
        SEMESTER_LABEL,
        COUNT(DISTINCT STUDENT_ID) AS COUNT
    FROM
        GRADES
        JOIN COURSES ON COURSES.COURSE_ID = GRADES.COURSE_ID
    GROUP BY COURSE_DEPT, COURSE_NUMBER, COURSE_TITLE, SEMESTER_LABEL
) COURSE_ENROLLMENT
GROUP BY COURSE_DEPT, COURSE_NUMBER, COURSE_TITLE;


> ### (9d)
> Write a query that returns, for every time each course was taught, the fraction of students enrolled in the course whose major matches the course's department.
> 
> Example output:
>> ```csv
>> dept,course_number,course_title,semester,frac_majoring
>> CSCI,3500,Cellular Automata, FA1995, 0.922
>> ASTR,2100,Planetary Motion, FA1995, 0.534
>> ```

In [ ]:
SELECT
    COURSE_DEPT,
    COURSE_NUMBER,
    COURSE_TITLE,
    SEMESTER_LABEL,
    N_STUDENTS_IN_DEPT::FLOAT/N_STUDENTS::FLOAT AS FRAC_MAJORING
FROM (
SELECT
    COURSE_DEPT,
    COURSE_NUMBER,
    COURSE_TITLE,
    SEMESTER_LABEL,
    COUNT(DISTINCT CASE WHEN MAJOR_GRADES.MAJOR = COURSE_DEPT THEN MAJOR_GRADES.STUDENT_ID END) AS N_STUDENTS_IN_DEPT,
    COUNT(DISTINCT MAJOR_GRADES.STUDENT_ID) AS N_STUDENTS
FROM
    (
    SELECT
        GRADES.SEMESTER_LABEL,
        GRADES.STUDENT_ID,
        GRADES.COURSE_ID,
        MAJORS.MAJOR
    FROM
        GRADES
        JOIN MAJORS ON GRADES.STUDENT_ID = MAJORS.STUDENT_ID AND GRADES.SEMESTER_LABEL = MAJORS.SEMESTER_LABEL
    ) MAJOR_GRADES
    JOIN COURSES ON COURSES.COURSE_ID = MAJOR_GRADES.COURSE_ID
GROUP BY COURSE_DEPT, COURSE_NUMBER, COURSE_TITLE, SEMESTER_LABEL
) STUDENT_COUNTS;

> ## Problem 10.
>
> ### (10a)
> 
> Students sometimes change their majors.
> 
> Write a query that returns, for each student, every major they ever had.
> - Include the *most recent* (aka latest) semester in which they had this major.
> 
> Example output:
>> ```csv
>> student_id,name,major,latest
>> N12345678,Alex Ample,MATH,FA2000
>> N12345678,Alex Ample,PHIL,SP2002
>> N12345678,Alex Ample,DATA,SP2004
>> N12345679,Bryce Bruin,CSCI,SP2003
>> ```

In [ ]:
SELECT
    STUDENTS.STUDENT_ID,
    STUDENTS.NAME,
    MAJOR_INFO.MAJOR,
    MAX(MAJOR_INFO.ACADEMIC_YEAR) AS LATEST
FROM (
    SELECT
        MAJORS.SEMESTER_LABEL,
        SEMESTERS.ACADEMIC_YEAR,
        MAJORS.STUDENT_ID,
        MAJORS.MAJOR
    FROM
        MAJORS
        JOIN SEMESTERS ON MAJORS.SEMESTER_LABEL = SEMESTERS.SEMESTER_LABEL
) MAJOR_INFO
    JOIN STUDENTS ON MAJOR_INFO.STUDENT_ID = STUDENTS.STUDENT_ID
GROUP BY STUDENTS.STUDENT_ID, STUDENTS.NAME, MAJOR_INFO.MAJOR;

> ### (10b)
> 
> Consider each student's *final* major - that is, the major they had during their last semester. We're interested in finding their average grade within that major.
> 
> Write a query that displays each student's final major and their average grade for courses *in that major*.
> 
> Example output:
> 
>> ```csv
>> student_id,name,major,avg_grade_in_major
>> N12345678,Alex Ample,DATA,0.78
>> N12345679,Bryce Bruin,CSCI,0.91
>> ```

In [2]:
SELECT
    STUDENT_ID,
    NAME,
    MAJOR,
    AVG(CASE WHEN MAJOR = COURSE_DEPT THEN GRADE END) AS AVERAGE_GRADE_IN_MAJOR
FROM (
SELECT
    LONG_INFO.*,
    STUDENTS.NAME
FROM (
SELECT
    GRADES_DEPT.STUDENT_ID,
    GRADES_DEPT.SEMESTER_LABEL,
    GRADES_DEPT.ACADEMIC_YEAR,
    MAJORS.MAJOR,
    GRADES_DEPT.COURSE_ID,
    GRADES_DEPT.COURSE_DEPT,
    GRADES_DEPT.GRADE
FROM (
    SELECT
        SEMESTER_LABEL,
        ACADEMIC_YEAR,
        STUDENT_ID,
        GRADES_YEAR.COURSE_ID,
        COURSES.COURSE_DEPT,
        GRADES_YEAR.GRADE
    FROM (
        SELECT
            GRADES.*,
            SEMESTERS.ACADEMIC_YEAR
        FROM GRADES
            JOIN SEMESTERS ON GRADES.SEMESTER_LABEL = SEMESTERS.SEMESTER_LABEL
    ) GRADES_YEAR
        JOIN COURSES ON GRADES_YEAR.COURSE_ID = COURSES.COURSE_ID
) GRADES_DEPT
    JOIN MAJORS ON MAJORS.SEMESTER_LABEL = GRADES_DEPT.SEMESTER_LABEL AND MAJORS.STUDENT_ID = GRADES_DEPT.STUDENT_ID
) LONG_INFO
    JOIN STUDENTS ON LONG_INFO.STUDENT_ID = STUDENTS.STUDENT_ID
) ALL_INFO
GROUP BY STUDENT_ID, NAME, MAJOR;

-- To complete this, I would need to select the major that occurred in the latest year.


IndentationError: unexpected indent (3760378102.py, line 2)

> ## (10c)
> 
> Suppose we wish to see *when* students chose to change their major.
> 
> Write a query that produces the following columns:
> - The student's ID,
> - their name,
> - the semester in which they entered a *new* major,
> - the major they switched *from*,
> - the major they switched *to*.
> 
> Students who did not change their major should not appear in the output.
> 
> Example output:
>> ```csv
>> student_id,name,semester,prev_major,next_major
>> N12345678,Alex Ample,SP2001,MATH,PHIL
>> N12345678,Alex Ample,FA2003,PHIL,DATA
>> ```



In [ ]:
SELECT
    PROFESSOR,
    SEMESTER_LABEL
FROM (
    SELECT DISTINCT
        PROFESSORS.PROFESSOR,
        GRADES.COURSE_ID,
        GRADES.SEMESTER_LABEL,
        GRADES.STUDENT_ID
    FROM
        GRADES
        JOIN PROFESSORS ON PROFESSORS.COURSE_ID = GRADES.COURSE_ID
) PROF_STUDENTS
GROUP BY PROFESSOR, SEMESTER_LABEL;

In [ ]:
SELECT 

FROM
    students JOIN housing ON students.student_id = housing.student_id
GROUP BY   
    housing.street_address,
    SELECT date_part('year', students.data_of_birth);